# Rakuten USA - Data Engineering Test

### Goals
To demonstrate your engineering skills. 

### Expectations
You can expect this test to use the following technologies:
- Python3
- Jupyter Notebook
- PySpark
- PySpark SQL

This test will cover the following areas:
- Dataset 
- Data cleaning/wrangling
- Exploratory Data Analysis (EDA)
- Data Visualization (Bonus)
- Lessons learned

### Where do you turn this test in?
Please email stirling.waite@rakuten with your finished notebook
- rakuten_engineering_test.ipynb
 
### Questions?
Feel free to email me or call me with any questions about this test
- stirling.waite@rakuten.com
- 801-856-1426

# About The Dataset

Below you will find a free Kaggle competition dataset from https://www.kaggle.com/mayank811/employee-reviews/data using scraped data from Glassdoor.com.

## Context
- Over 67k employee reviews for Google, Amazon, Facebook, Apple, Microsoft, and Netflix.
- Web scraped from Glassdoor

## Content
This dataset contains employee reviews separated into the following categories:

- **Index:** index
- **Company:** Company name
- **Location:** This dataset is global, as such it may include the country's name in parenthesis [i.e "Toronto, ON(Canada)"]. However, if the location is in the USA then it will only include the city and state[i.e "Los Angeles, CA" ]
- **Date Posted:** in the following format MM DD, YYYY
- **Job-Title:** This string will also include whether the reviewer is a 'Current' or 'Former' Employee at the time of the review
- **Summary:** Short summary of employee review
- **Pros:** Pros
- **Cons:** Cons
- **Overall Rating:** 1-5
- **Work/Life Balance Rating:** 1-5
- **Culture and Values Rating:** 1-5
- **Career Opportunities Rating:** 1-5
- **Comp & Benefits Rating:** 1-5
- **Senior Management Rating:** 1-5
- **Helpful Review Count:** 1-5

**Link to Review:** This will provide you with a direct link to the page that contains the review. However it is likely that this link will be outdated

**NOTE:** 'none' is placed in all cells where no data value was found.



In [133]:
import re
import pyspark
from pyspark import SparkConf, SparkContext, SQLContext
from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import *
from pyspark.sql import functions as fn
from pyspark.sql import types as t
from pyspark.sql.types import *
from datetime import datetime

In [2]:
conf = SparkConf()\
                .setMaster("local[*]")\
                .setAppName("rakutenHomework")
        
sc = SparkContext(conf=conf)
sc.setLogLevel("ERROR")
sqlContext = SQLContext(sc)
spark = SparkSession.builder.appName("spark play").getOrCreate()

In [31]:
reviews = spark.read.csv("employee_reviews.csv", header=True, mode="DROPMALFORMED", inferSchema='true', encoding="utf-8")
reviews.show()

+---+-------+-----------------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------+------------------+--------------------+--------------------------+------------------+----------------------+-------------+--------------------+
|_c0|company|         location|        dates|           job-title|             summary|                pros|                cons|      advice-to-mgmt|overall-ratings|work-balance-stars|culture-values-stars|carrer-opportunities-stars|comp-benefit-stars|senior-mangemnet-stars|helpful-count|                link|
+---+-------+-----------------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------+------------------+--------------------+--------------------------+------------------+----------------------+-------------+--------------------+
|  1| google|             none| Dec 11, 2018|Current Employee ...|B

# Data Cleaning & Removing Un-needed Columns

No one likes dirty data.  Let's clean up this dataset to make it more usable for our use case.

 
1. Make the date field more data friendly
    - Hint #1: Use the provided `review_date_udf` udf provided (it will make your life more simple) 
2. Using the "location" column 
    - Add new columns for city, state, country columns
        - Hint #1: If location is not in US it will show as Toronto, ON(Canada)
        - Hint #2: Use something like the following to extract the `country` column
            - ```fn.when(~col("location").contains("("), "USA").otherwise(regexp_extract(col("location"), "\((.*?)\)", 1)).alias("country")```
3. Using the "job-title" column  
    - Add a new boolean column titled "current_employee" that will show:
        - 1,  if reviewer is a current employee
        - 0,  if reviewer is not a current employee
    - Remove "Former Employee - " and "" from the "job-title" column
        - Hint: Use the udf function `reg_job_title_udf`
4. Drop un-needed columns 
    - "_c0", "link", "pros", "cons", "advice-to-mgmt", "summary"

##### Useful UDF functions needed for below

In [165]:
review_date_udf = fn.udf(
    lambda x: datetime.strptime(x, ' %b %d, %Y'), DateType()
)

In [170]:
def reg_job_title(column):
    res_split=[]
    reg_patterns=["Former Employee - / ", "Current Employee - / "]
    for i in range(len(reg_patterns)):
        res_split=re.findall(r"[^/]+",reg_patterns[i])
        for x in res_split[0].split("|"):
            column = column.replace(x,res_split[1])
    return column[1:]

reg_job_title_udf = fn.udf(reg_job_title, t.StringType())

In [206]:
new_reviews_df = reviews.where((col("dates") != 'None') & (col("dates") != ' Jan 0, 0000') & (col("dates") != ' Nov 0, 0000')) \
                         .select(
                            col("company"),
                            col("location"),
                            fn.when(~col("location").contains("("), "USA").otherwise(regexp_extract(col("location"), "\((.*?)\)", 1)).alias("country"),
                            review_date_udf(col('dates')).alias("review_date"),
                            reg_job_title_udf(col("job-title")).alias("job_title"),
                            fn.when(col("job-title").contains('Current'), 1).otherwise(0).alias("current_employee"),
                            col("overall-ratings").alias("overall_ratings"),
                            col("work-balance-stars").alias("work_balance_stars"),
                            col("culture-values-stars").alias("culture_values_stars"),
                            col("carrer-opportunities-stars").alias("carrer_opportunities_stars"),
                            col("comp-benefit-stars").alias("comp_benefit_stars"),
                            col("senior-mangemnet-stars").alias("senior_mangemnet_stars"),
                            col("helpful-count").alias("helpful_count")
                         )

new_reviews_df.count()
new_reviews_df.show()

+-------+-----------------+-------+-----------+--------------------+----------------+---------------+------------------+--------------------+--------------------------+------------------+----------------------+-------------+
|company|         location|country|review_date|           job_title|current_employee|overall_ratings|work_balance_stars|culture_values_stars|carrer_opportunities_stars|comp_benefit_stars|senior_mangemnet_stars|helpful_count|
+-------+-----------------+-------+-----------+--------------------+----------------+---------------+------------------+--------------------+--------------------------+------------------+----------------------+-------------+
| google|             none|    USA| 2018-12-11|  Anonymous Employee|               1|            5.0|               4.0|                 5.0|                       5.0|               4.0|                   5.0|            0|
| google|Mountain View, CA|    USA| 2015-02-08|  Anonymous Employee|               1|            5.0

In [208]:
new_reviews_df.createOrReplaceTempView("reviews")

### (1) What is the total number of reviews in the data set?

##### PySpark Implmentation

In [215]:
new_reviews_df.count()

64623

##### Spark SQL Implmentation

In [210]:
spark.sql("SELECT count(*) as review_count from reviews").show()


+------------+
|review_count|
+------------+
|       64623|
+------------+



###  (2) How many reviews per each company?

##### PySpark Implmentation

In [216]:
company_reviews = new_reviews_df.select("company").groupBy('company').count().orderBy('count', ascending=[0])
company_reviews.show()

+---------+-----+
|  company|count|
+---------+-----+
|   amazon|25335|
|microsoft|17150|
|    apple|12397|
|   google| 7552|
| facebook| 1492|
|  netflix|  697|
+---------+-----+



##### Spark SQL Implmentation

In [214]:
spark.sql("""SELECT 
                company,
                count(*) as review_count 
             from reviews
             group by company
             order by review_count DESC
           """
         )\
         .show()

+---------+------------+
|  company|review_count|
+---------+------------+
|   amazon|       25335|
|microsoft|       17150|
|    apple|       12397|
|   google|        7552|
| facebook|        1492|
|  netflix|         697|
+---------+------------+



###  (3) How many reviews (former and current) per each company are in the dataset?

##### PySpark Implmentation

In [217]:
company_current_reviews = new_reviews_df.select(
                                        col("company"), 
                                        col("current_employee")
                                     ) \
                                     .groupBy(['company', 'current_employee']) \
                                     .count() \
                                     .orderBy(['company','count'])
company_current_reviews.show()

+---------+----------------+-----+
|  company|current_employee|count|
+---------+----------------+-----+
|   amazon|               0| 8391|
|   amazon|               1|16944|
|    apple|               0| 5373|
|    apple|               1| 7024|
| facebook|               0|  288|
| facebook|               1| 1204|
|   google|               0| 3011|
|   google|               1| 4541|
|microsoft|               0| 6292|
|microsoft|               1|10858|
|  netflix|               0|  342|
|  netflix|               1|  355|
+---------+----------------+-----+



##### Spark SQL Implmentation

In [219]:
spark.sql("""SELECT 
                company,
                current_employee,
                count(*) as review_count 
             from reviews
             group by company, current_employee
             order by company, review_count ASC
           """
         )\
         .show()

+---------+----------------+------------+
|  company|current_employee|review_count|
+---------+----------------+------------+
|   amazon|               0|        8391|
|   amazon|               1|       16944|
|    apple|               0|        5373|
|    apple|               1|        7024|
| facebook|               0|         288|
| facebook|               1|        1204|
|   google|               0|        3011|
|   google|               1|        4541|
|microsoft|               0|        6292|
|microsoft|               1|       10858|
|  netflix|               0|         342|
|  netflix|               1|         355|
+---------+----------------+------------+



###  (4) What is the average review scores of each current and former employees for each company?

##### PySpark Implmentation

In [185]:
company_current_avg_review_scores = new_reviews.select(
                                                col("company"), 
                                                col("current_employee"),
                                                col("overall_ratings")
                                             ) \
                                             .groupBy(['company', 'current_employee']) \
                                             .agg(avg(col("overall_ratings")).alias("avg_overall_rating")) \
                                             .orderBy(['company', 'avg_overall_rating'], ascending=[1,0])
company_current_avg_review_scores.show()

+---------+----------------+------------------+
|  company|current_employee|avg_overall_rating|
+---------+----------------+------------------+
|   amazon|               1|  3.81952313503305|
|   amazon|               0|3.2210701942557503|
|    apple|               1| 4.088126423690205|
|    apple|               0| 3.868416154848316|
| facebook|               1| 4.679401993355482|
| facebook|               0| 3.888888888888889|
|   google|               1|4.4441752917859505|
|   google|               0| 4.244104948522086|
|microsoft|               1|3.9408730889666606|
|microsoft|               0| 3.690241576605213|
|  netflix|               1| 3.963380281690141|
|  netflix|               0| 3.043859649122807|
+---------+----------------+------------------+



##### Spark SQL Implmentation

In [221]:
spark.sql("""SELECT 
                company,
                current_employee,
                avg(overall_ratings) AS avg_overall_rating
             from reviews
             group by company, current_employee
             order by company, avg_overall_rating DESC
           """
         )\
         .show()

+---------+----------------+------------------+
|  company|current_employee|avg_overall_rating|
+---------+----------------+------------------+
|   amazon|               1|  3.81952313503305|
|   amazon|               0|3.2210701942557503|
|    apple|               1| 4.088126423690205|
|    apple|               0| 3.868416154848316|
| facebook|               1| 4.679401993355482|
| facebook|               0| 3.888888888888889|
|   google|               1|4.4441752917859505|
|   google|               0| 4.244104948522086|
|microsoft|               1|3.9408730889666606|
|microsoft|               0| 3.690241576605213|
|  netflix|               1| 3.963380281690141|
|  netflix|               0| 3.043859649122807|
+---------+----------------+------------------+



###  (5) Show the following:
1. Former Google employee reviewers 
2. For locations in Utah, San Francisco, or Japan
3. Show count and average work_balance_score 

##### PySpark Implmentation

In [205]:
reviews_by_country = new_reviews.select(
                                    col("company"),
                                    col("location"),
                                    col("country"),
                                    col("current_employee"),
                                    col("work_balance_stars")
                                ) \
                                .where(((col("current_employee").contains(0))) & ((col("location").contains("UT")) | (col("location").contains("San Francisco")) | (col("Country") == "Japan"))) \
                                .groupBy(["company", "location", "country"]) \
                                .agg(
                                    fn.count(fn.lit(1)).alias("review_count"),
                                    avg(col("work_balance_stars")).alias("work_balance_stars_avg")
                                ) \
                                .orderBy(['work_balance_stars_avg', 'review_count'], ascending=[0, 0])

reviews_by_country.show()

+---------+--------------------+-------+------------+----------------------+
|  company|            location|country|review_count|work_balance_stars_avg|
+---------+--------------------+-------+------------+----------------------+
|   amazon|Ichikawa, Chiba (...|  Japan|           2|                   5.0|
|microsoft|          Murray, UT|    USA|           2|                   5.0|
|microsoft|         Redmond, UT|    USA|           2|                   5.0|
|    apple|       Bountiful, UT|    USA|           1|                   5.0|
|    apple|          Murray, UT|    USA|           1|                   5.0|
|   google|           Kanab, UT|    USA|           1|                   5.0|
|   google|   San Francisco, CA|    USA|          72|     4.355072463768116|
|   google|       Tokyo (Japan)|  Japan|           5|                   4.2|
|    apple|  Salt Lake City, UT|    USA|          15|     4.153846153846154|
|microsoft|    Hachiōji (Japan)|  Japan|           1|                   4.0|

##### Spark SQL Implmentation

In [224]:
spark.sql("""SELECT 
                company,
                location,
                country,
                count(*) AS review_count,
                avg(work_balance_stars) AS work_balance_stars_avg
             from reviews
             WHERE current_employee = 0
             group by company, location, country
             order by work_balance_stars_avg, review_count DESC
           """
         )\
         .show()

+---------+--------------------+-------+------------+----------------------+
|  company|            location|country|review_count|work_balance_stars_avg|
+---------+--------------------+-------+------------+----------------------+
|    apple|       Hollywood, CA|    USA|           2|                  null|
|microsoft|       Shoreline, WA|    USA|           1|                  null|
|   amazon|        Ringgold, GA|    USA|           1|                  null|
|   amazon|     Kansas City, MO|    USA|           1|                  null|
|   amazon|  Sparrows Point, MD|    USA|           1|                  null|
|microsoft|Palm Beach Garden...|    USA|           1|                  null|
|    apple|Longueuil, QC (Ca...| Canada|           1|                  null|
|microsoft|      Indore (India)|  India|           1|                  null|
|   amazon|   Overland Park, KS|    USA|           1|                  null|
|   amazon|         Pomfret, MD|    USA|           1|                  null|

# Visualize Your Data (Bonus Points)